## Bloomberg BQuant Webinar Series: <br> Finding ESG Bonds
This is a companion notebook to the "Seeing Green: Finding ESG Bonds" webinar.

In [ ]:
import bql
import pandas as pd
import bqviz as bqv
from functools import reduce

In [ ]:
bq = bql.Service()

In [ ]:
equity = '324382z Ln Equity'

vm_bonds = bq.univ.debt(equity)

request = bql.Request(vm_bonds,bq.data.name().group().count())

response = bq.execute(request)[0].df()
response

In [ ]:
bond = bq.univ.list('AM2044296 Corp')

#direct_issuer = bq.univ.issuerof(bond)
#parent = bq.univ.parent(bond,type='direct')
ultimate_parent = bq.univ.parent(bond,type='ultimate')

request = bql.Request(ultimate_parent,bq.data.name())

response = bq.execute(request)

response[0].df()

# Linking Bonds to ESG/Fundamental Data

In [ ]:
bond_index = bq.univ.members('LP01TREU Index')

issuer_univ = bq.univ.issuerof(bond_index)

issuer_count = bq.data.net_debt_to_ebitda().group(bq.data.gics_sector_name()).median()

request = bql.Request(issuer_univ,issuer_count)

response = bq.execute(request)

response[0].df()

# ESG Ranking

In [ ]:
bonds_univ = bq.univ.members('LUACTRUU Index')

issuer_univ = bq.univ.parent(bonds_univ,type='Ultimate')

In [ ]:
#esg factors

ghg = bq.data.ghg_intensity_per_sales(fpt='A',FPO='-1')
energy = bq.data.energy_intensity_per_sales(fpt='a',fpo='-1')

ghg_rank = ghg.group().dropna(remove_id=True).rank(order='asc')
energy_rank = energy.group().dropna(remove_id=True).rank(order='asc')

top_10_esg_score = bq.func.matches(energy_rank,energy_rank<=10)

fields = {'Score':top_10_esg_score.groupsort(order='asc')}

In [ ]:
request = bql.Request(issuer_univ,fields,with_params={'currency':'USD'})

In [ ]:
response = bq.execute(request)
response[0].df()

In [ ]:
bonds_for_issuer = bq.univ.bonds('PPG US Equity')

fields = {'Green Bond':bq.data.green_bond_loan_indicator(),
         'Spread':bq.data.spread(),
         'Maturity':bq.data.maturity()}

request = bql.Request(bonds_for_issuer,fields)

response = bq.execute(request)

chart_data = pd.concat([r.df().iloc[:,[-1]] for r in response],axis=1)
chart_data.head()

In [ ]:
scatter = bqv.InteractiveScatterPlot(chart_data[['Maturity','Spread']])
scatter.show()

In [ ]:
scatter.selected_data

# Green Bond Issuance

In [ ]:
total_univ = bq.univ.bondsuniv('All')

conditions = [bq.data.issue_dt() >='2015-01-01',
             bq.data.issue_dt()<='2019-09-30',
             bq.data.green_bond_loan_indicator()=='Y']

criteria = reduce(bq.func.and_,conditions)

greenbonds_univ = bq.univ.filter(total_univ,criteria)

#data
amt_issued = bq.data.amt_issued(currency='USD').group(bq.data.issue_dt().year()).sum()

fields = {'Amt Issued (USD)': amt_issued}

request = bql.Request(greenbonds_univ,fields)
response = bq.execute(request)

final_output = response[0].df()[['Amt Issued (USD)']]
final_output

In [ ]:
bqv.BarPlot(final_output,color_mode='index').show()

In [ ]:
total_univ = bq.univ.bondsuniv('All')

conditions = [bq.data.issue_dt() >='2015-01-01',
             bq.data.issue_dt()<='2019-09-30',
             bq.data.green_bond_loan_indicator()=='Y',
             bq.data.bb_composite() >= 'BBB-']

criteria = reduce(bq.func.and_,conditions)

greenbonds_univ = bq.univ.filter(total_univ,criteria)

#data
amt_issued = bq.data.amt_issued(currency='USD')
issuer_name = bq.data.long_comp_name()
amt_by_issuer = amt_issued.group(issuer_name).sum().groupsort()/10000000000

request = bql.Request(greenbonds_univ,{'Amt (USD)':amt_by_issuer})

response = bq.execute(request)
response[0].df().head()

In [ ]:
print(criteria)